In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sqlite3 as sq
pd.options.mode.chained_assignment = None

In [2]:
def mantissa_round(x):
    y = np.floor(x)
    indices = np.argsort(x-y)
    hilowman = indices[::-1]
    diffxy = np.sum(x)-np.sum(y)
    z=0
    while z < int(diffxy):
        y[hilowman[z]] +=1
        z +=1
    return np.array(y, dtype='int')

In [22]:
dist = pd.read_csv('../data/LEHDDistribution.csv')
dist['County'] = dist['County'].str.split(pat = ',', expand = True)[0]
dist['County'] = dist['County'] + ", Tennessee"
dist = dist.rename(columns = {'County': 'NAME'})
dist.head(2)

,GEO_ID,NAME,Total Jobs %,"Agriculture, forestry, fishing and hunting %","Mining, quarrying, and oil and gas extraction %",Utilities %,Construction %,Manufacturing %,Wholesale trade %,Retail trade %,Transportation and warehousing %,Information %,Finance and insurance %,Real estate and rental and leasing %,"Professional, scientific, and technical services %",Management of companies and enterprises %,Administrative and support and waste management and remediation services %,Educational services %,Health care and social assistance %,"Arts, entertainment, and recreation %",Accommodation and food services %,Other services (except government and government enterprises) %,Public Administration %
0,470210701021001,"Cheatham County, Tennessee",0.009725,0.0,0.0,0.0,0.004242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.115318,0.0,0.0,0.0,0.020305
1,470210701021004,"Cheatham County, Tennessee",0.002093,0.0,0.0,0.0,0.010604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022222,0.0,0.0,0.0,0.001721,0.0,0.0,0.0,0.000000


In [23]:
dist['GEO_ID'] = dist['GEO_ID'].astype(str)
c_inverted = dict(zip(dist['GEO_ID'], dist['NAME']))

In [24]:
bl = pd.read_csv('../data/allyears2digmantissadround.csv')
bl = bl.loc[bl['Year'] == '2023']
bl.drop(columns = 'Year', inplace = True)
bl = bl.set_index('Industry').transpose()
bl = bl.add_suffix(' County Total').reset_index(drop = False).rename(columns = {'index': 'NAME'})
bl.head()

Industry,NAME,Accommodation and food services County Total,Administrative and support and waste management and remediation services County Total,"Agriculture, forestry, fishing and hunting County Total","Arts, entertainment, and recreation County Total",Construction County Total,Educational services County Total,Finance and insurance County Total,Health care and social assistance County Total,Information County Total,Management of companies and enterprises County Total,Manufacturing County Total,"Mining, quarrying, and oil and gas extraction County Total",Other services (except government and government enterprises) County Total,"Professional, scientific, and technical services County Total",Public Administration County Total,Real estate and rental and leasing County Total,Retail trade County Total,Transportation and warehousing County Total,Utilities County Total,Wholesale trade County Total,Total employment (number of jobs) County Total
0,"Cheatham County, Tennessee",982.0,1062.0,487.0,585.0,2003.0,525.0,590.0,830.0,217.0,126.0,2929.0,18.0,1154.0,812.0,1216.0,913.0,1493.0,1064.0,55.0,232.0,17292.000000
1,"Davidson County, Tennessee",63676.0,64965.0,683.0,33160.0,41425.0,39871.0,45164.0,99949.0,21871.0,16631.0,22708.0,706.0,38795.0,67880.0,25701.0,40991.0,51251.0,51385.0,687.0,25987.0,753485.500000
2,"Dickson County, Tennessee",2246.0,2239.0,1269.0,455.0,2650.0,411.0,1106.0,3213.0,227.0,105.0,3689.0,88.0,1751.0,882.0,2264.0,1271.0,3142.0,907.0,38.0,786.0,28738.750000
3,"Houston County, Tennessee",147.0,69.0,411.0,8.0,256.0,22.0,69.0,343.0,22.0,1.0,211.0,1.0,191.0,57.0,477.0,57.0,218.0,91.0,3.0,8.0,2661.178571
4,"Humphreys County, Tennessee",724.0,268.0,699.0,75.0,890.0,166.0,253.0,732.0,40.0,1.0,1563.0,100.0,530.0,218.0,856.0,231.0,961.0,532.0,27.0,152.0,9017.821429


In [25]:
data = bl.merge(dist, on = 'NAME', how = 'outer')

In [26]:
industries = ['Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']
for ind in industries:
    data['{}'.format(ind)] = data['{} County Total'.format(ind)] * data['{} %'.format(ind)]

In [27]:
data.head()

,NAME,Accommodation and food services County Total,Administrative and support and waste management and remediation services County Total,"Agriculture, forestry, fishing and hunting County Total","Arts, entertainment, and recreation County Total",Construction County Total,Educational services County Total,Finance and insurance County Total,Health care and social assistance County Total,Information County Total,Management of companies and enterprises County Total,Manufacturing County Total,"Mining, quarrying, and oil and gas extraction County Total",Other services (except government and government enterprises) County Total,"Professional, scientific, and technical services County Total",Public Administration County Total,Real estate and rental and leasing County Total,Retail trade County Total,Transportation and warehousing County Total,Utilities County Total,Wholesale trade County Total,Total employment (number of jobs) County Total,GEO_ID,Total Jobs %,"Agriculture, forestry, fishing and hunting %","Mining, quarrying, and oil and gas extraction %",Utilities %,Construction %,Manufacturing %,Wholesale trade %,Retail trade %,Transportation and warehousing %,Information %,Finance and insurance %,Real estate and rental and leasing %,"Professional, scientific, and technical services %",Management of companies and enterprises %,Administrative and support and waste management and remediation services %,Educational services %,Health care and social assistance %,"Arts, entertainment, and recreation %",Accommodation and food services %,Other services (except government and government enterprises) %,Public Administration %,"Agriculture, forestry, fishing and hunting","Mining, quarrying, and oil and gas extraction",Utilities,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,Finance and insurance,Real estate and rental and leasing,"Professional, scientific, and technical services",Management of companies and enterprises,Administrative and support and waste management and remediation services,Educational services,Health care and social assistance,"Arts, entertainment, and recreation",Accommodation and food services,Other services (except government and government enterprises),Public Administration
0,"Cheatham County, Tennessee",982.0,1062.0,487.0,585.0,2003.0,525.0,590.0,830.0,217.0,126.0,2929.0,18.0,1154.0,812.0,1216.0,913.0,1493.0,1064.0,55.0,232.0,17292.0,470210701021001,0.009725,0.0,0.0,0.0,0.004242,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.115318,0.0,0.0,0.0,0.020305,0.0,0.0,0.0,8.496288,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,95.714286,0.0,0.0,0.0,24.690355
1,"Cheatham County, Tennessee",982.0,1062.0,487.0,585.0,2003.0,525.0,590.0,830.0,217.0,126.0,2929.0,18.0,1154.0,812.0,1216.0,913.0,1493.0,1064.0,55.0,232.0,17292.0,470210701021004,0.002093,0.0,0.0,0.0,0.010604,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.022222,0.0,0.000000,0.0,0.001721,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,21.240721,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,18.044444,0.0,0.00000,0.0,1.428571,0.0,0.0,0.0,0.000000
2,"Cheatham County, Tennessee",982.0,1062.0,487.0,585.0,2003.0,525.0,590.0,830.0,217.0,126.0,2929.0,18.0,1154.0,812.0,1216.0,913.0,1493.0,1064.0,55.0,232.0,17292.0,470210701021009,0.000369,0.0,0.0,0.0,0.001060,0.0,0.0,0.0011,0.002564,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,2.124072,0.0,0.0,1.642464,2.728205,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,"Cheatham County, Tennessee",982.0,1062.0,487.0,585.0,2003.0,525.0,590.0,830.0,217.0,126.0,2929.0,18.0,1154.0,812.0,1216.0,913.0,1493.0,1064.0,55.0,232.0,17292.0,470210701021012,0.000246,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.007407,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,6.014815,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000
4,"Cheatham County, Tennessee",982.0,1062.0,487.0,585.0,2003.0,

In [29]:
data = data.rename(columns = {'NAME': 'County'})
data = data[['County', 'GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]

In [37]:
cheatham = data.loc[data['County'] == 'Cheatham County, Tennessee']
davidson = data.loc[data['County'] == 'Davidson County, Tennessee']
dickson = data.loc[data['County'] == 'Dickson County, Tennessee']
houston = data.loc[data['County'] == 'Houston County, Tennessee']
humphreys = data.loc[data['County'] == 'Humphreys County, Tennessee']
maury = data.loc[data['County'] == 'Maury County, Tennessee']
montgomery = data.loc[data['County'] == 'Montgomery County, Tennessee']
robertson = data.loc[data['County'] == 'Robertson County, Tennessee']
rutherford = data.loc[data['County'] == 'Rutherford County, Tennessee']
stewart = data.loc[data['County'] == 'Stewart County, Tennessee']
sumner = data.loc[data['County'] == 'Sumner County, Tennessee']
trousdale = data.loc[data['County'] == 'Trousdale County, Tennessee']
williamson = data.loc[data['County'] == 'Williamson County, Tennessee']
wilson = data.loc[data['County'] == 'Wilson County, Tennessee']

In [38]:
# dfs = [cheatham, davidson, dickson, houston, humphreys, maury, montgomery, robertson, rutherford, stewart, sumner, trousdale, williamson, wilson]
# inds = ['Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 'Manufacturing', 
#         'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 'Real estate and rental and leasing', 
#         'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#         'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#         'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#         'Public Administration']
# for df in dfs:
#     data = df
#     data = data[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
#               'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
#               'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#               'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#               'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#               'Public Administration']]
#     for ind in inds:
#         data['{} Mantissa'.format(ind)] = mantissa_round(data['{}'.format(ind)])
#         df = data

In [39]:
# cheatham = cheatham[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
#               'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
#               'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#               'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#               'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#               'Public Administration']]
# for ind in inds:
#     cheatham['{} Mantissa'.format(ind)] = mantissa_round(data['{}'.format(ind)])

In [40]:
cheatham = cheatham[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
cheatham.set_index('GEO_ID', inplace = True)
inds = list(cheatham.columns)
cheatham.reset_index(drop = False, inplace = True)
cheatham.fillna(0, inplace = True)
for ind in inds:
    cheatham['{} Mantissa'.format(ind)] = mantissa_round(cheatham['{}'.format(ind)])
cheatham['County'] = 'Cheatham County, Tennessee'

In [41]:
diff = cheatham.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff1 = diff.reset_index(drop = False)
diff1.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Cheatham County, Tennessee",0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [42]:
davidson = davidson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
davidson.set_index('GEO_ID', inplace = True)
inds = list(davidson.columns)
davidson.reset_index(drop = False, inplace = True)
davidson.fillna(0, inplace = True)
for ind in inds:
    davidson['{} Mantissa'.format(ind)] = mantissa_round(davidson['{}'.format(ind)])
davidson['County'] = 'Davidson County, Tennessee'

In [43]:
diff = davidson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff2 = diff.reset_index(drop = False)
diff2.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Davidson County, Tennessee",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [44]:
dickson = dickson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
dickson.set_index('GEO_ID', inplace = True)
inds = list(dickson.columns)
dickson.reset_index(drop = False, inplace = True)
dickson.fillna(0, inplace = True)
for ind in inds:
    dickson['{} Mantissa'.format(ind)] = mantissa_round(dickson['{}'.format(ind)])
dickson['County'] = 'Dickson County, Tennessee'

In [45]:
diff = dickson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff3 = diff.reset_index(drop = False)
diff3.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Dickson County, Tennessee",1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [46]:
houston = houston[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
houston.set_index('GEO_ID', inplace = True)
inds = list(houston.columns)
houston.reset_index(drop = False, inplace = True)
houston.fillna(0, inplace = True)
for ind in inds:
    houston['{} Mantissa'.format(ind)] = mantissa_round(houston['{}'.format(ind)])
houston['County'] = 'Houston County, Tennessee'

In [47]:
diff = houston.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = (diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)])
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff4 = diff.reset_index(drop = False)
diff4.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Houston County, Tennessee",1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,-1.421085e-14,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [48]:
humphreys = humphreys[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
humphreys.set_index('GEO_ID', inplace = True)
inds = list(humphreys.columns)
humphreys.reset_index(drop = False, inplace = True)
humphreys.fillna(0, inplace = True)
for ind in inds:
    humphreys['{} Mantissa'.format(ind)] = mantissa_round(humphreys['{}'.format(ind)])
humphreys['County'] = 'Humphreys County, Tennessee'

In [49]:
diff = humphreys.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff5 = diff.reset_index(drop = False)
diff5.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Humphreys County, Tennessee",0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [50]:
maury = maury[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
maury.set_index('GEO_ID', inplace = True)
inds = list(maury.columns)
maury.reset_index(drop = False, inplace = True)
maury.fillna(0, inplace = True)
for ind in inds:
    maury['{} Mantissa'.format(ind)] = mantissa_round(maury['{}'.format(ind)])
maury['County'] = 'Maury County, Tennessee'

In [51]:
diff = maury.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff6 = diff.reset_index(drop = False)
diff6.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Maury County, Tennessee",1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [52]:
montgomery = montgomery[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
montgomery.set_index('GEO_ID', inplace = True)
inds = list(montgomery.columns)
montgomery.reset_index(drop = False, inplace = True)
montgomery.fillna(0, inplace = True)
for ind in inds:
    montgomery['{} Mantissa'.format(ind)] = mantissa_round(montgomery['{}'.format(ind)])
montgomery['County'] = 'Montgomery County, Tennessee'

In [53]:
diff = montgomery.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff7 = diff.reset_index(drop = False)
diff7.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Montgomery County, Tennessee",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [54]:
robertson = robertson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
robertson.set_index('GEO_ID', inplace = True)
inds = list(robertson.columns)
robertson.reset_index(drop = False, inplace = True)
robertson.fillna(0, inplace = True)
for ind in inds:
    robertson['{} Mantissa'.format(ind)] = mantissa_round(robertson['{}'.format(ind)])
robertson['County'] = 'Robertson County, Tennessee'

In [55]:
diff = robertson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff8 = diff.reset_index(drop = False)
diff8.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Robertson County, Tennessee",1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [56]:
rutherford = rutherford[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
rutherford.set_index('GEO_ID', inplace = True)
inds = list(rutherford.columns)
rutherford.reset_index(drop = False, inplace = True)
rutherford.fillna(0, inplace = True)
for ind in inds:
    rutherford['{} Mantissa'.format(ind)] = mantissa_round(rutherford['{}'.format(ind)])
rutherford['County'] = 'Rutherford County, Tennessee'

In [57]:
diff = rutherford.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff9 = diff.reset_index(drop = False)
diff9.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Rutherford County, Tennessee",1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [58]:
stewart = stewart[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
stewart.set_index('GEO_ID', inplace = True)
inds = list(stewart.columns)
stewart.reset_index(drop = False, inplace = True)
stewart.fillna(0, inplace = True)
for ind in inds:
    stewart['{} Mantissa'.format(ind)] = mantissa_round(stewart['{}'.format(ind)])
stewart['County'] = 'Stewart County, Tennessee'

In [59]:
diff = stewart.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff10 = diff.reset_index(drop = False)
diff10.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Stewart County, Tennessee",1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,-7.105427e-15,1.0,0.0,1.0,1.0,1.0


In [60]:
sumner = sumner[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
sumner.set_index('GEO_ID', inplace = True)
inds = list(sumner.columns)
sumner.reset_index(drop = False, inplace = True)
sumner.fillna(0, inplace = True)
for ind in inds:
    sumner['{} Mantissa'.format(ind)] = mantissa_round(sumner['{}'.format(ind)])
sumner['County'] = 'Sumner County, Tennessee'

In [61]:
diff = sumner.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff11 = diff.reset_index(drop = False)
diff11.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Sumner County, Tennessee",1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [62]:
trousdale = trousdale[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
trousdale.set_index('GEO_ID', inplace = True)
inds = list(trousdale.columns)
trousdale.reset_index(drop = False, inplace = True)
trousdale.fillna(0, inplace = True)
for ind in inds:
    trousdale['{} Mantissa'.format(ind)] = mantissa_round(trousdale['{}'.format(ind)])
trousdale['County'] = 'Trousdale County, Tennessee'

In [63]:
diff = trousdale.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff12 = diff.reset_index(drop = False)
diff12.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Trousdale County, Tennessee",0.0,0.0,1.0,1.0,-5.684342e-14,1.0,1.0,1.0,-3.552714e-15,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0


In [64]:
williamson = williamson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
williamson.set_index('GEO_ID', inplace = True)
inds = list(williamson.columns)
williamson.reset_index(drop = False, inplace = True)
williamson.fillna(0, inplace = True)
for ind in inds:
    williamson['{} Mantissa'.format(ind)] = mantissa_round(williamson['{}'.format(ind)])
williamson['County'] = 'Williamson County, Tennessee'

In [65]:
diff = williamson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff13 = diff.reset_index(drop = False)
diff13.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Williamson County, Tennessee",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.818989e-12


In [66]:
wilson = wilson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
wilson.set_index('GEO_ID', inplace = True)
inds = list(wilson.columns)
wilson.reset_index(drop = False, inplace = True)
wilson.fillna(0, inplace = True)
for ind in inds:
    wilson['{} Mantissa'.format(ind)] = mantissa_round(wilson['{}'.format(ind)])
wilson['County'] = 'Wilson County, Tennessee'

In [67]:
diff = wilson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff14 = diff.reset_index(drop = False)
diff14.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,"Wilson County, Tennessee",0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [68]:
diffs = [diff1, diff2, diff3, diff4, diff5, diff6, diff7, diff8, diff9, diff10, diff11, diff12, diff13, diff14]
diff = pd.concat(diffs)

In [69]:
diff.to_csv('diffs2023.csv')

In [70]:
dfs = [cheatham, davidson, dickson, houston, humphreys, maury, montgomery, robertson, rutherford, stewart, sumner, trousdale, williamson, wilson]
data = pd.concat(dfs)

In [71]:
data['GEO_ID'] = data['GEO_ID'].astype(str)
#data['County'] = data['GEO_ID'].map(c_inverted)

In [72]:
data = data[['County', 'GEO_ID', 'Agriculture, forestry, fishing and hunting Mantissa', 'Mining, quarrying, and oil and gas extraction Mantissa', 'Utilities Mantissa',
             'Construction Mantissa', 
              'Manufacturing Mantissa', 'Wholesale trade Mantissa', 'Retail trade Mantissa', 'Transportation and warehousing Mantissa', 'Information Mantissa', 
             'Finance and insurance Mantissa', 
              'Real estate and rental and leasing Mantissa', 'Professional, scientific, and technical services Mantissa', 
             'Management of companies and enterprises Mantissa', 
              'Administrative and support and waste management and remediation services Mantissa', 'Educational services Mantissa', 
             'Health care and social assistance Mantissa', 
              'Arts, entertainment, and recreation Mantissa', 'Accommodation and food services Mantissa', 
             'Other services (except government and government enterprises) Mantissa', 
              'Public Administration Mantissa']]

In [73]:
data['GEO_ID'] = 'G'+ data['GEO_ID']

In [74]:
data.head()

,County,GEO_ID,"Agriculture, forestry, fishing and hunting Mantissa","Mining, quarrying, and oil and gas extraction Mantissa",Utilities Mantissa,Construction Mantissa,Manufacturing Mantissa,Wholesale trade Mantissa,Retail trade Mantissa,Transportation and warehousing Mantissa,Information Mantissa,Finance and insurance Mantissa,Real estate and rental and leasing Mantissa,"Professional, scientific, and technical services Mantissa",Management of companies and enterprises Mantissa,Administrative and support and waste management and remediation services Mantissa,Educational services Mantissa,Health care and social assistance Mantissa,"Arts, entertainment, and recreation Mantissa",Accommodation and food services Mantissa,Other services (except government and government enterprises) Mantissa,Public Administration Mantissa
0,"Cheatham County, Tennessee",G470210701021001,1,0,1,8,1,1,1,1,1,1,1,1,0,1,1,95,1,1,1,24
1,"Cheatham County, Tennessee",G470210701021004,0,0,0,21,0,0,0,0,0,0,0,18,0,0,0,1,0,0,0,0
2,"Cheatham County, Tennessee",G470210701021009,0,0,0,2,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0
3,"Cheatham County, Tennessee",G470210701021012,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0
4,"Cheatham County, Tennessee",G470210701021013,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0


In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12040 entries, 0 to 885
Data columns (total 22 columns):
 #   Column                                                                             Non-Null Count  Dtype 
---  ------                                                                             --------------  ----- 
 0   County                                                                             12040 non-null  object
 1   GEO_ID                                                                             12040 non-null  object
 2   Agriculture, forestry, fishing and hunting Mantissa                                12040 non-null  int32 
 3   Mining, quarrying, and oil and gas extraction Mantissa                             12040 non-null  int32 
 4   Utilities Mantissa                                                                 12040 non-null  int32 
 5   Construction Mantissa                                                              12040 non-null  int32 
 6   Manuf

In [77]:
data.to_csv('mantissad_2023_blocks.csv')